# 5.- Test Local Training

In [5]:
import os
import docker

In [6]:
# Define the path to the directory containing your Dockerfile.
# If you're running on Windows, make sure to use the appropriate path format.
dockerfile_path = "../docker"  # Adjust this path as needed

In [7]:
# Initialize the Docker client (make sure Docker Desktop is running on Windows)
client = docker.from_env()

# Define a tag for the built image.
image_tag = "satellite-object-detection-local:latest"

In [10]:
try:
    # Build the Docker image.
    # 'path' specifies the directory with the Dockerfile.
    # 'tag' sets the name and tag for the resulting image.
    # 'rm=True' removes intermediate containers after a successful build.
    image, build_logs = client.images.build(
        path=dockerfile_path,
        tag=image_tag,
        rm=True
    )

    # Print the build logs to monitor the build process.
    for chunk in build_logs:
        if 'stream' in chunk:
            print(chunk['stream'].strip())

    print("Image built successfully. Tags:", image.tags)

except docker.errors.BuildError as build_err:
    # Catch and print errors that occur during the build process.
    print("Error building the image:", build_err)
except Exception as err:
    # Catch any other exceptions that might occur.
    print("An error occurred:", err)


Error building the image: The command '/bin/sh -c echo "Estoy en:" $(pwd) && pip3 install /models/research/' returned a non-zero code: 1


In [2]:
!wsl --shutdown